In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
df_green = spark.read.parquet('./data/green_tripdata_2021-01')

In [ ]:
rdd = df_green.select('lpep_pickup_datetime', 'PULocationID', 'total_amount').rdd
rdd.take(5)

In [ ]:
from datetime import datetime

def filter_outliers(row):
    return row.lpep_pickup_datetime >= datetime(year=2020, month=1, day=1)

In [ ]:
rdd = rdd.filter(filter_outliers)
rdd.take(10)

In [ ]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)

In [ ]:
rdd = rdd.map(prepare_for_grouping)
rdd.take(10)

In [ ]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value 
      
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count) 

In [ ]:
rdd = rdd.reduceByKey(calculate_revenue)
rdd.take(10)

In [ ]:
from collections import namedtuple
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [ ]:
rdd = rdd.map(unwrap)
rdd.take(10)

In [ ]:
from pyspark.sql import types
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [ ]:
df_result = rdd.toDF(result_schema)
df_result.show()

In [ ]:
df_result.write.parquet('./data/report/revenue/green_rdd')

Map Partition

In [2]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']
df_green = spark.read.parquet('./data/green_tripdata_2021-01')
duration_rdd = df_green.select(columns).rdd
duration_rdd.take(10)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2021, 1, 27, 9, 10, 53), PULocationID=43, DOLocationID=229, trip_distance=3.09),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2021, 1, 9, 7, 18, 37), PULocationID=244, DOLocationID=240, trip_distance=6.13),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2021, 1, 16, 12, 56, 9), PULocationID=25, DOLocationID=33, trip_distance=1.02),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2021, 1, 18, 20, 31, 5), PULocationID=95, DOLocationID=95, trip_distance=1.83),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2021, 1, 2, 16, 39, 30), PULocationID=82, DOLocationID=95, trip_distance=2.17),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2021, 1, 22, 14, 4, 28), PULocationID=166, DOLocationID=142, trip_distance=2.55),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2021, 1, 11, 20, 24, 52), PULocationID=97, DOLocationID=89, trip_distance=4.2),
 Row(VendorID=2, lpep_pickup_datetime=dateti

In [3]:
import pyspark.pandas as pd

def apply_model_in_batch(partition):
    df = pd.DataFrame(partition, columns=columns)
    cnt = len(df)
    return [cnt]

d:\data-engineering-zoomcamp\.venv\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [4]:
duration_rdd.mapPartitions(apply_model_in_batch).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 1 times, most recent failure: Lost task 1.0 in stage 2.0 (TID 3) (LAPTOP-66MED8N2 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\data-engineering-zoomcamp\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\sql\pandas\utils.py", line 27, in require_minimum_pandas_version
    import pandas
ModuleNotFoundError: No module named 'pandas'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "D:\data-engineering-zoomcamp\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1227, in main
  File "D:\data-engineering-zoomcamp\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 90, in read_command
  File "D:\data-e